In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib

# Charger les données et processing
credits_data = pd.read_csv("../data/credits.csv")
titles_data = pd.read_csv("../data/titles.csv")

titles_data["title"].fillna("Unknown", inplace=True)

# Processisng (normalisation ... ) ....


# Utils
def get_color_for_score(score, num_colors=10, color_palette="coolwarm"):
    """
    Retourne une couleur correspondant à la note du film.
    """
    cmap = plt.get_cmap(color_palette, num_colors)
    if pd.isna(score) or score < 0 or score > 10:
        return "#808080"
    color_index = int(score * (num_colors - 1) / 10)
    return matplotlib.colors.rgb2hex(cmap(color_index))


def filter_titles_by_score(titles_data, credits_data, selected_scores):
    # Calculer la note moyenne pour chaque film
    titles_data["average_score"] = titles_data[["imdb_score", "tmdb_score"]].mean(
        axis=1
    )
    # Filtrer les films dont la note moyenne est égale à la note sélectionnée
    filtered_titles = pd.DataFrame()
    for n in selected_scores:
        # Filtrer les lignes pour cet intervalle spécifique
        temp_df = titles_data[
            (titles_data["average_score"] >= n)
            & (titles_data["average_score"] <= n + 1)
        ]

        # Ajouter les résultats au DataFrame filtré
        filtered_titles = pd.concat([filtered_titles, temp_df])

    filtered_titles = filtered_titles.drop_duplicates()
    # titles_data["rounded_average_score"] = titles_data["average_score"].round()
    # condition = titles_data["rounded_average_score"].isin(selected_scores)

    # filtered_titles = titles_data[condition]

    # Filtrer les données de crédits pour inclure uniquement les films sélectionnés
    return credits_data[credits_data["id"].isin(filtered_titles["id"])]


def create_movie_network_with_color(
    credits_data,
    titles_data,
    node_size=10,
    color_palette="coolwarm",
    selected_scores=None,
):
    """
    Crée un graphe de réseau pour les films connectés par des acteurs communs et attribue une couleur
    aux nœuds en fonction de la note moyenne du film.

    Args:
    credits_data (pd.DataFrame): DataFrame contenant les données des acteurs et des films.
    titles_data (pd.DataFrame): DataFrame contenant les informations sur les films.
    node_size (int): Taille des nœuds dans le graphe.

    Returns:
    plotly.graph_objs._figure.Figure: Un objet Figure Plotly représentant le graphe de réseau.
    """

    if selected_scores is not None and len(selected_scores) > 0:
        # Filtrer les films en fonction de la note
        credits_data = filter_titles_by_score(
            titles_data, credits_data, selected_scores
        )
    # Création d'un dictionnaire pour associer chaque acteur à ses films
    actor_to_movies = {}
    for index, row in credits_data.iterrows():
        actor_to_movies.setdefault(row["name"], []).append(row["id"])

    # Création d'un graphe vide
    G = nx.Graph()

    # Ajout des nœuds (films) et des arêtes (relations entre films basées sur des acteurs communs)
    for actor, movies in actor_to_movies.items():
        for i in range(len(movies)):
            for j in range(i + 1, len(movies)):
                G.add_edge(movies[i], movies[j])

    # Positionnement des nœuds
    pos = nx.spring_layout(G)

    # Création des arêtes
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    # Création des nœuds
    node_x = []
    node_y = []
    node_text = []
    node_color = []
    for node in G.nodes():
        x, y = pos[node]
        # Calculer la note moyenne du film
        film_data = titles_data[titles_data["id"] == node]
        if not film_data.empty:
            imdb_score = film_data["imdb_score"].values[0]
            tmdb_score = film_data["tmdb_score"].values[0]
            avg_score = (
                (imdb_score + tmdb_score) / 2
                if pd.notnull(imdb_score) and pd.notnull(tmdb_score)
                else None
            )

            node_text.append(f"{film_data['title'].values[0]} - Avg Score: {avg_score}")
            # Attribuer une couleur en fonction de la note
            node_color.append(
                get_color_for_score(avg_score, color_palette=color_palette)
            )
        else:
            node_text.append("Unknown Film")
            node_color.append("#808080")  # Gris pour les films inconnus
        node_x.append(x)
        node_y.append(y)

    # Création de la figure Plotly
    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=0.5, color="#888"),
        hoverinfo="none",
        mode="lines",
    )
    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers",
        hoverinfo="text",
        text=node_text,
        marker=dict(showscale=False, size=node_size, color=node_color, line_width=2),
    )

    # Création de la figure finale
    fig = go.Figure(
        data=[edge_trace, node_trace],
        layout=go.Layout(
            title="<br>Réseau de Films basé sur des Acteurs Communs",
            titlefont_size=16,
            showlegend=False,
            hovermode="closest",
            margin=dict(b=20, l=5, r=5, t=40),
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        ),
    )

    return fig


app = dash.Dash(__name__)


def filter_titles_per_year(titles_data, credits_data, year):
    # Filtrer les films dont la note moyenne est égale à la note sélectionnée
    filtered_titles = pd.DataFrame()
    # Filtrer les lignes pour cet intervalle spécifique
    temp_df = titles_data[
        (titles_data["year"] >= year) & (titles_data["year"] <= year + 10)
    ]
    # Filtrer les résultats pour avoir seulement les 10 premiers
    temp_df = temp_df.sort_values(by=["average_score"], ascending=False).head(10)

    # Ajouter les résultats au DataFrame filtré
    filtered_titles = pd.concat([filtered_titles, temp_df])

    filtered_titles = filtered_titles.drop_duplicates()
    # Filtrer les données de crédits pour inclure uniquement les films sélectionnés
    actors = credits_data[credits_data["id"].isin(filtered_titles["id"])]
    return actors, filtered_titles


title_category_dict = {
    "all_time": -1,
    "2020": 2020,
    "2010": 2010,
    "2000": 2000,
    "20th": 1900,
}

title_selector = [
    {
        "label": "Top 10 des titres les mieux notés sur IMDB et TMDB ALL-TIME",
        "value": title_category_dict["all_time"],
    },
    {
        "label": "Top 10 des titres les mieux notés sur IMDB et TMDB années 2020",
        "value": title_category_dict["2020"],
    },
    {
        "label": "Top 10 des titres les mieux notés sur IMDB et TMDB années 2010",
        "value": title_category_dict["2010"],
    },
    {
        "label": "Top 10 des titres les mieux notés sur IMDB et TMDB années 2000",
        "value": title_category_dict["2000"],
    },
    {
        "label": "Top 10 des titres les mieux notés sur IMDB et TMDB du 20ième siècle",
        "value": title_category_dict["20th"],
    },
]

# Layout de l'application
app.layout = html.Div(
    [
        html.H1("Réseau de Films/Séries basé sur des Acteurs Communs"),
        html.H2("Paramètres"),
        html.H3("Taille des noeuds"),
        dcc.Slider(
            id="node-size-slider",
            min=10,
            max=50,
            step=10,
            value=20,
        ),
        html.H3(
            "Top 10 de tous les titres par décennies/siècles selon leurs notes obtenus sur IMDB/TMDB"
        ),
        dcc.Dropdown(
            id="title-selection-dropdown",
            options=title_selector,
            multi=False,
        ),
        html.H3(
            "Acteurs à inclure (une fois sélectionné, la liste est re-calculé avec seulement les acteurs en commun)"
        ),
        dcc.Dropdown(
            id="actor-selector",
            options=[
                {"label": actor, "value": actor}
                for actor in credits_data["name"].unique()
            ],
            multi=True,
        ),
        html.H3(
            "Palettes de couleurs (pour les films/séries sans note, la couleur grise est utilisée)"
        ),
        dcc.Dropdown(
            id="color-palette-selector",
            options=[
                {"label": "Viridis", "value": "viridis"},
                {"label": "Plasma", "value": "plasma"},
                {"label": "Inferno", "value": "inferno"},
                {"label": "Magma", "value": "magma"},
                {"label": "Cividis", "value": "cividis"},
                {"label": "Coolwarm", "value": "coolwarm"},
                {"label": "Spectral", "value": "Spectral"},
                {"label": "PiYG", "value": "PiYG"},
                {"label": "PRGn", "value": "PRGn"},
                {"label": "Twilight", "value": "twilight"},
                {"label": "HSV", "value": "hsv"},
                {"label": "Pastel1", "value": "Pastel1"},
                {"label": "Set3", "value": "Set3"},
                {"label": "Tab10", "value": "tab10"},
                {"label": "Accent", "value": "Accent"},
            ],
            value="viridis",  # Valeur par défaut
        ),
        html.H3(
            id="score-selector-title",
            title="Filtrer les films/séries par note (les films/séries sans note ne sont pas affichés)",
        ),
        dcc.Dropdown(
            id="score-selector",
            options=[{"label": str(score), "value": score} for score in range(1, 11)],
            multi=True,
            placeholder="Sélectionnez une note",
        ),
        dcc.Graph(id="movie-network-graph"),
    ]
)


@app.callback(
    Output("score-selector", "style"),
    Output("score-selector-title", "style"),
    [Input("actor-selector", "value"), Input("title-selection-dropdown", "value")],
)
def toggle_score_selector(selected_actors, selected_title_selection):
    if selected_actors or selected_title_selection:
        return {"display": "block"}, {"display": "block"}
    else:
        return {"display": "none"}, {"display": "none"}


# Callback pour mettre à jour le graphe en fonction du slider et de la sélection d'acteurs
@app.callback(
    Output("movie-network-graph", "figure"),
    [
        Input("node-size-slider", "value"),
        Input("title-selection-dropdown", "value"),
        Input("actor-selector", "value"),
        Input("color-palette-selector", "value"),
        Input("score-selector", "value"),
    ],
)
def update_graph(
    node_size, title_selection, selected_actors, color_palette, selected_scores
):
    if title_selection:
        fig = create_movie_network_with_color(
            filtered_credits_data,
            titles_data,
            node_size,
            color_palette=color_palette,
            selected_scores=selected_scores,
        )
    elif selected_actors:
        selected_movies = credits_data[credits_data["name"].isin(selected_actors)][
            "id"
        ].unique()
        filtered_credits_data = credits_data[credits_data["id"].isin(selected_movies)]
        fig = create_movie_network_with_color(
            filtered_credits_data,
            titles_data,
            node_size,
            color_palette=color_palette,
            selected_scores=selected_scores,
        )
    else:
        fig = create_movie_network_with_color(
            credits_data,
            titles_data,
            node_size,
            color_palette=color_palette,
            selected_scores=selected_scores,
        )
    return fig


# Lancer l'application
if __name__ == "__main__":
    app.run_server(debug=True)